In [2]:
import numpy as np
import nltk
import pandas as pd
import string
from sklearn import metrics
import re
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import PredefinedSplit
from sklearn.feature_extraction.text import CountVectorizer
import dill
import pickle

In [3]:
class Data_set:
    
    def __init__ (self):
        self.vocab = []
        self.X_train = []
        self.y_train = []
        self.X_test = []
        self.y_test = []
        self.X_valid = []
        self.y_valid = []
    
    
    def process(self, raw, vocab):
        translator = str.maketrans('', '', string.punctuation)
        raw = raw.translate(translator)
        raw = raw.lower()
        if(vocab == 0):
            raw = raw.replace("\t", " ")
        raw = raw.replace("\t0\n", " ")
        raw = raw.replace("\t1\n", " ")
        raw = raw.replace("\t2\n", " ")
        raw = raw.replace("\t3\n", " ")
        raw = raw.replace("\t4\n", " ")
        raw = raw.replace("\t5\n", " ")
        return raw
    
    def load(self, filename, dset, train):
            X = []
            y = []
            if(train == 1):
                self.vocab.append(self.process(open(filename).read(), 1))
            if(dset == 'yelp'):
                lines = open(filename).read().splitlines()
            if(dset == 'imdb'):
                lines = re.split("(?<=[0-1])[\n]*", open(filename).read())
            for x in lines:
                X.append(x[:-1])
                y.append(x[-1:])
            if(dset == 'imdb'):
                X = X[:-1]
                y = y[:-1]
            X_p = []
            for i in range(len(X)):
                X_p.append(self.process(X[i], 0))
            return X_p, y
    
    def imp(self, filetrain, filevalid, filetest, dset):
        self.X_train, self.y_train = self.load(filetrain, dset, 1)
        self.X_valid, self.y_valid = self.load(filevalid, dset, 0)
        self.X_test, self.y_test = self.load(filetest, dset, 0)

In [4]:
yelp = Data_set()
yelp.imp('hwk3_datasets/yelp-train.txt', 'hwk3_datasets/yelp-valid.txt', 'hwk3_datasets/yelp-test.txt', 'yelp')
imdb = Data_set()
imdb.imp('hwk3_datasets/IMDB-train.txt', 'hwk3_datasets/IMDB-valid.txt', 'hwk3_datasets/IMDB-test.txt', 'imdb')

/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [5]:
vectorizer = CountVectorizer(max_features = 10000)
vector_train = vectorizer.fit_transform(yelp.X_train)
vector_valid = vectorizer.transform(yelp.X_valid)
vector_test = vectorizer.transform(yelp.X_test)
vector = vectorizer.transform(yelp.vocab)

freq = vector.toarray()
words_freq = [(word, idx, freq[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
#words_freq = words_freq[0:10000]
#with open('yelp-vocab.txt', 'w') as fp: 
#    fp.write('\n'.join('%s\t%s\t%s' % x for x in words_freq))
#words_freq

In [6]:
words = []
vector = vectorizer.fit_transform(yelp.X_train)
array = vector.toarray()
for j in range(len(yelp.X_train)):
    words.append([])
    for i in range(10000):
        if(array[j][i] != 0):
            words[j].append([i, array[j][i]])

In [367]:
words = []
vector = vectorizer.transform(yelp.X_test)
array = vector.toarray()
for j in range(len(yelp.X_test)):
    words.append([])
    for i in range(10000):
        if(array[j][i] != 0):
            words[j].append([i, array[j][i]])

In [108]:
#f = open("yelp-test.txt","w+")
#for i in range(len(words)):
#    for y in words[i]:
#        f.write('%s\t' % str(y[0]))
#    f.write(yelp.y_test[i])
#    f.write("\n")

# Testing

In [94]:
y_train = yelp.y_train
y_valid = yelp.y_valid
y_test = yelp.y_test

## BBOW

In [95]:
bbow = np.zeros(shape=(7000,10000), dtype=np.int64)
for i in range(len(words)):
    for j in range(len(words[i])):
            bbow[i][words[i][j][0]] = 1

### Baseline

In [255]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='uniform').fit(bbow, y_train)
y_pr = clf.predict(vector_test)
clf = DummyClassifier(strategy='most_frequent').fit(bbow, y_train)
y_pred = clf.predict(vector_test.toarray())
print(metrics.f1_score(y_test, y_pr, average='micro'))
print(metrics.f1_score(y_test, y_pred, average='micro'))

0.2005
0.351


### BernoulliNB

In [311]:
from sklearn.naive_bayes import BernoulliNB 
tuned_parameters = [{'alpha': [0.01, 1, 0.5, 0.2, 0.1]}]
n_folds = 5
clf = BernoulliNB()
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(bbow, y_train)
print(clf.best_estimator_)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': [0.01, 1, 0.5, 0.2, 0.1]}],
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn', scoring='f1_micro', verbose=0)

In [318]:
print(clf.best_estimator_)
y_pr = clf.predict(vector_train.toarray())
print(metrics.f1_score(y_train, y_pr, average='micro'))
y_pr = clf.predict(vector_valid.toarray())
print(metrics.f1_score(y_valid, y_pr, average='micro'))
y_pr = clf.predict(vector_test.toarray())
print(metrics.f1_score(y_test, y_pr, average='micro'))

BernoulliNB(alpha=0.01, binarize=0.0, class_prior=None, fit_prior=True)
0.7482857142857143
0.425
0.437


### Decision Tree

In [323]:
from sklearn.tree import DecisionTreeClassifier
tuned_parameters = [{'max_depth': [1, 2, 10, 32], 'min_samples_split': [0.1, 0.3, 0.7, 1.0]}]
n_folds = 3
clf = DecisionTreeClassifier()
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(bbow, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [1, 2, 10, 32], 'min_samples_split': [0.1, 0.3, 0.7, 1.0]}],
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn', scoring='f1_micro', verbose=0)

In [324]:
print(clf.best_estimator_)
y_pr = clf.predict(vector_train.toarray())
print(metrics.f1_score(y_train, y_pr, average='micro'))
y_pr = clf.predict(vector_valid.toarray())
print(metrics.f1_score(y_valid, y_pr, average='micro'))
y_pr = clf.predict(vector_test.toarray())
print(metrics.f1_score(y_test, y_pr, average='micro'))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=0.1,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.43257142857142855
0.38499999999999995
0.405


### Linear SVM

In [96]:
from sklearn.svm import LinearSVC
tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1, 10]}]
n_folds = 3
clf = LinearSVC(random_state=0, max_iter=5000)
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(bbow, y_train)
print(clf.best_estimator_)

/usr/local/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=5000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)


In [99]:
y_pr = clf.predict(vector_train.toarray())
print("Linear SVM BBoW train", metrics.f1_score(y_train, y_pr, average='micro'))

Linear SVM BBoW train 0.5515714285714286


In [100]:
y_pr = clf.predict(vector_valid.toarray())
print("Linear SVM BBoW valid", metrics.f1_score(y_valid, y_pr, average='micro'))

Linear SVM BBoW valid 0.452


In [101]:
y_pr = clf.predict(vector_test.toarray())
print("Linear SVM BBoW test", metrics.f1_score(y_test, y_pr, average='micro'))

Linear SVM BBoW test 0.4755


## FBOW

In [102]:
fbow = np.zeros(shape=(7000,10000))
array = vector_train.toarray()
for i in range(len(words)):
    sum_w = np.sum(words[i], axis=0)
    for j in range(len(words[i])):
            fbow[i][words[i][j][0]] = array[i][words[i][j][0]] / sum_w[1]

### Baseline

In [228]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='uniform', random_state=0).fit(fbow, y_train)
y_pr = clf.predict(vector_test)
clf = DummyClassifier(strategy='most_frequent', random_state=0).fit(fbow, y_train)
y_pred = clf.predict(vector_test)
print(metrics.f1_score(y_test, y_pr, average='micro'))
print(metrics.f1_score(y_test, y_pred, average='micro'))

0.189
0.351


### GaussianNB

In [328]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB().fit(fbow, y_train)
y_pr = clf.predict(vector_train.toarray())
print(metrics.f1_score(y_train, y_pr, average='micro'))
y_pr = clf.predict(vector_valid.toarray())
print(metrics.f1_score(y_valid, y_pr, average='micro'))
y_pr = clf.predict(vector_test.toarray())
print(metrics.f1_score(y_test, y_pr, average='micro'))

0.9385714285714286
0.356
0.3495


### Decision Tree

In [329]:
from sklearn.tree import DecisionTreeClassifier
tuned_parameters = [{'max_depth': [1, 2, 10, 32], 'min_samples_split': [0.1, 0.3, 0.7, 1.0]}]
n_folds = 3
clf = DecisionTreeClassifier()
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(fbow, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [1, 2, 10, 32], 'min_samples_split': [0.1, 0.3, 0.7, 1.0]}],
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn', scoring='f1_micro', verbose=0)

In [330]:
print(clf.best_estimator_)
y_pr = clf.predict(vector_train.toarray())
print(metrics.f1_score(y_train, y_pr, average='micro'))
y_pr = clf.predict(vector_valid.toarray())
print(metrics.f1_score(y_valid, y_pr, average='micro'))
y_pr = clf.predict(vector_test.toarray())
print(metrics.f1_score(y_test, y_pr, average='micro'))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=32,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=0.1,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.4308571428571429
0.408
0.4144999999999999


### Linear SVM

In [104]:
from sklearn.svm import LinearSVC
tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1, 10]}]
n_folds = 3
clf = LinearSVC(random_state=0, max_iter=5000)
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(fbow, y_train)
print(clf.best_estimator_)

LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=5000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)


In [105]:
y_pr = clf.predict(vector_train.toarray())
print("Linear SVM FBoW train", metrics.f1_score(y_train, y_pr, average='micro'))

Linear SVM FBoW train 0.6435714285714286


In [106]:
y_pr = clf.predict(vector_valid.toarray())
print("Linear SVM FBoW valid", metrics.f1_score(y_valid, y_pr, average='micro'))

Linear SVM FBoW valid 0.456


In [107]:
y_pr = clf.predict(vector_test.toarray())
print("Linear SVM FBoW test", metrics.f1_score(y_test, y_pr, average='micro'))

Linear SVM FBoW test 0.461


# IMDB

In [338]:
#vector = vectorizer.transform(imdb.vocab)
#freq = vector.toarray()
#words_freq = [(word, idx, freq[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
#words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
#words_freq = words_freq[0:10000]
#with open('imdb-vocab.txt', 'w') as fp: 
#    fp.write('\n'.join('%s\t%s\t%s' % x for x in words_freq))
#words_freq

In [5]:
def get_words(dset):
    words = []
    vector = vectorizer.transform(dset)
    array = vector.toarray()
    for j in range(len(dset)):
        words.append([])
        for i in range(10000):
            if(array[j][i] != 0):
                words[j].append([i, array[j][i]])
    return words

In [16]:
#f = open("IMDB-test.txt","w+")
#for i in range(len(words)):
#    for y in words[i]:
#        f.write('%s\t' % str(y[0]))
#    f.write(imdb.y_test[i])
#    f.write("\n")

In [52]:
vectorizer = CountVectorizer(max_features = 10000)
vector_train = vectorizer.fit_transform(imdb.X_train)
vector_valid = vectorizer.transform(imdb.X_valid)
vector_test = vectorizer.transform(imdb.X_test)

In [8]:
words_imdb_train = get_words(imdb.X_train)
words_imdb_valid = get_words(imdb.X_valid)
words_imdb_test = get_words(imdb.X_test)

In [66]:
imdb_X_comb = []
for i in range(len(imdb.X_train)):
    imdb_X_comb.append(imdb.X_train[i])
for i in range(len(imdb.X_valid)):
    imdb_X_comb.append(imdb.X_valid[i])
words_imdb_comb = get_words(imdb_X_comb)

In [50]:
bbow = np.zeros(shape=(15000,10000), dtype=np.int64)
for i in range(len(words_imdb_train)):
    for j in range(len(words_imdb_train[i])):
            bbow[i][words_imdb_train[i][j][0]] = 1

In [53]:
fbow = np.zeros(shape=(15000,10000))
array = vector_train.toarray()
for i in range(len(words_imdb_train)):
    sum_w = np.sum(words_imdb_train[i], axis=0)
    for j in range(len(words_imdb_train[i])):
            fbow[i][words_imdb_train[i][j][0]] = array[i][words_imdb_train[i][j][0]] / sum_w[1]

In [64]:
imdb_y_comb = []
for i in range(len(imdb.y_train)):
    imdb_y_comb.append(imdb.y_train[i])
for i in range(len(imdb.y_valid)):
    imdb_y_comb.append(imdb.y_valid[i])

In [54]:
y_train = imdb.y_train
y_valid = imdb.y_valid
y_test = imdb.y_test

In [40]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='uniform').fit(bbow, y_train)
y_pr = clf.predict(vector_test)
print("Uniform BBoW: ", metrics.f1_score(y_test, y_pr, average='micro'))
clf = DummyClassifier(strategy='uniform').fit(fbow, y_train)
y_pr = clf.predict(vector_test)
print("Uniform FBoW: ", metrics.f1_score(y_test, y_pr, average='micro'))

Uniform BBoW:  0.49972
Majority BBoW:  0.5
Uniform FBoW:  0.49508
Majority FBoW:  0.5


### NB

In [76]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
clf = BernoulliNB(alpha=0.2).fit(bbow, y_train)
y_pr = clf.predict(vector_train.toarray())
print("BernoulliNB BBoW train: ", metrics.f1_score(y_train, y_pr, average='micro'))

BernoulliNB BBoW train:  0.8729999999999999


In [77]:
y_pr = clf.predict(vector_valid.toarray())
print("BernoulliNB BBoW valid: ", metrics.f1_score(y_valid, y_pr, average='micro'))

BernoulliNB BBoW valid:  0.8425


In [79]:
y_pr = clf.predict(vector_test.toarray())
print("BernoulliNB BBoW test: ", metrics.f1_score(y_test, y_pr, average='micro'))

BernoulliNB BBoW test:  0.8354


In [84]:
clf = GaussianNB().fit(fbow, y_train)
y_pr = clf.predict(vector_train.toarray())
print("GaussianNB FBoW train: ", metrics.f1_score(y_train, y_pr, average='micro'))

GaussianNB FBoW train:  0.8744666666666666


In [85]:
y_pr = clf.predict(vector_valid.toarray())
print("GaussianNB FBoW valid: ", metrics.f1_score(y_valid, y_pr, average='micro'))

GaussianNB FBoW valid:  0.7753


In [86]:
y_pr = clf.predict(vector_test.toarray())
print("GaussianNB FBoW test: ", metrics.f1_score(y_test, y_pr, average='micro'))

GaussianNB FBoW test:  0.72016


### Decision Tree

In [79]:
from sklearn.tree import DecisionTreeClassifier

In [83]:
tuned_parameters = [{'max_depth': [1, 2, 10, 32], 'min_samples_split': [0.1, 0.3, 0.7, 1.0]}]
n_folds = 3
clf = DecisionTreeClassifier()
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(bbow, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [1, 2, 10, 32], 'min_samples_split': [0.1, 0.3, 0.7, 1.0]}],
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn', scoring='f1_micro', verbose=0)

In [84]:
print(clf.best_estimator_)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=32,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=0.1,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [85]:
y_pr = clf.predict(vector_train.toarray())
print("Decision Tree BBoW train: ", metrics.f1_score(y_train, y_pr, average='micro'))

Decision Tree BBoW train:  0.7412666666666666


In [86]:
y_pr = clf.predict(vector_valid.toarray())
print("Decision Tree BBoW valid: ", metrics.f1_score(y_valid, y_pr, average='micro'))

Decision Tree BBoW valid:  0.7249


In [87]:
y_pr = clf.predict(vector_test.toarray())
print("Decision Tree BBoW test: ", metrics.f1_score(y_test, y_pr, average='micro'))

Decision Tree BBoW test:  0.72824


In [89]:
tuned_parameters = [{'max_depth': [1, 2, 10, 32], 'min_samples_split': [0.1, 0.3, 0.7, 1.0]}]
n_folds = 3
clf = DecisionTreeClassifier()
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(fbow, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [1, 2, 10, 32], 'min_samples_split': [0.1, 0.3, 0.7, 1.0]}],
       pre_dispatch='2*n_jobs', refit='f1_micro',
       return_train_score='warn', scoring='f1_micro', verbose=0)

In [90]:
print(clf.best_estimator_)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=32,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=0.1,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [91]:
y_pr = clf.predict(vector_train.toarray())
print("Decision Tree BBoW train: ", metrics.f1_score(y_train, y_pr, average='micro'))

Decision Tree BBoW train:  0.7203333333333334


In [92]:
y_pr = clf.predict(vector_valid.toarray())
print("Decision Tree BBoW valid: ", metrics.f1_score(y_valid, y_pr, average='micro'))

Decision Tree BBoW valid:  0.7172


In [93]:
y_pr = clf.predict(vector_test.toarray())
print("Decision Tree BBoW test: ", metrics.f1_score(y_test, y_pr, average='micro'))

Decision Tree BBoW test:  0.7148


### Linear SVM

In [13]:
from sklearn import svm

In [ ]:
tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1, 10]}]
n_folds = 3
clf = LinearSVC(random_state=0, max_iter=5000)
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(bbow, y_train)
print(clf.best_estimator_)

In [64]:
clf = LinearSVC(random_state=0, max_iter=5000, C=0.01).fit(bbow, y_train)

In [65]:
y_pr = clf.predict(vector_train.toarray())
print("Linear SVM BBoW train: ", metrics.f1_score(y_train, y_pr, average='micro'))

Linear SVM BBoW train:  0.935


In [66]:
y_pr = clf.predict(vector_valid.toarray())
print("Linear SVM BBoW valid: ", metrics.f1_score(y_valid, y_pr, average='micro'))

Linear SVM BBoW valid:  0.8668


In [67]:
y_pr = clf.predict(vector_test.toarray())
print("Linear SVM BBoW test: ", metrics.f1_score(y_test, y_pr, average='micro'))

Linear SVM BBoW test:  0.85844


In [73]:
tuned_parameters = [{'C': [10, 20, 50]}]
n_folds = 3
clf = LinearSVC(random_state=0, max_iter=5000)
clf = GridSearchCV(clf, tuned_parameters, scoring='f1_micro', cv=n_folds, refit='f1_micro')
clf.fit(fbow, y_train)
print(clf.best_estimator_)

LinearSVC(C=50, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=5000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)


In [74]:
clf = LinearSVC(random_state=0, max_iter=5000, C=50).fit(fbow, y_train)

In [75]:
y_pr = clf.predict(vector_train.toarray())
print("Linear SVM FBoW train: ", metrics.f1_score(y_train, y_pr, average='micro'))

Linear SVM FBoW train:  0.8496


In [76]:
y_pr = clf.predict(vector_valid.toarray())
print("Linear SVM FBoW valid: ", metrics.f1_score(y_valid, y_pr, average='micro'))

Linear SVM FBoW valid:  0.8078


In [77]:
y_pr = clf.predict(vector_test.toarray())
print("Linear SVM FBoW test: ", metrics.f1_score(y_test, y_pr, average='micro'))

Linear SVM FBoW test:  0.8056799999999998
